In [2]:
import torch
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import numpy as np
from matplotlib import pyplot as plt
import time
import cv2 as cv

In [3]:
trainds = torchvision.datasets.CIFAR10(
    root = "./data",
    train = True,
    transform = torchvision.transforms.ToTensor(),  
    download = True
)

testds = torchvision.datasets.CIFAR10(
    root = "./data",
    train = False,
    transform = torchvision.transforms.ToTensor(), 
    download = True
)

batch_size = 100

trainldr = torch.utils.data.DataLoader(
    dataset = trainds,
    batch_size = batch_size,
    shuffle = True
)

testldr = torch.utils.data.DataLoader(
    dataset = testds,
    batch_size = batch_size
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
img_hwc = np.array(trainds[0][0] * 255).astype(np.uint8).transpose((1, 2, 0))
img_hwc = cv.resize(img_hwc, (100, 100), interpolation = cv.INTER_LINEAR)

#plt.imshow(img_hwc)

In [5]:
'''class cnn(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.mxpul = torch.nn.MaxPool2d((2, 2), stride = 2)

        self.convlay0 = torch.nn.Conv2d(3, 64, 3, stride = 1, padding = 'same')
        self.convlay1 = torch.nn.Conv2d(64, 64, 3, stride = 1, padding = 'same')
        self.convlay2 = torch.nn.Conv2d(64, 128, 3, stride = 1, padding = 'same')
        self.convlay3 = torch.nn.Conv2d(128, 128, 3, stride = 1, padding = 'same')
        self.convlay4 = torch.nn.Conv2d(128, 256, 3, stride = 1, padding = 'same')
        self.convlay5 = torch.nn.Conv2d(256, 256, 3, stride = 1, padding = 'same')
        self.convlay6 = torch.nn.Conv2d(256, 512, 3, stride = 1, padding = 'same')
        self.convlay7 = torch.nn.Conv2d(512, 512, 3, stride = 1, padding = 'same')
        self.convlay8 = torch.nn.Conv2d(512, 512, 3, stride = 1, padding = 'same')

        self.linlay0 = torch.nn.Linear(512 * 4, 1024)
        self.linlay1 = torch.nn.Linear(1024, 1024)
        self.linlay2 = torch.nn.Linear(1024, 10)
    
    def forward(self, x):

        out = F.relu(self.convlay0(x))
        out = self.mxpul(F.relu(self.convlay1(out)))
        out = F.relu(self.convlay2(out))
        out = self.mxpul(F.relu(self.convlay3(out)))
        out = F.relu(self.convlay4(out))
        out = self.mxpul(F.relu(self.convlay5(out)))
        out = F.relu(self.convlay6(out))
        out = F.relu(self.convlay7(out))
        out = self.mxpul(F.relu(self.convlay8(out)))
        
        out = out.view(-1, 512 * 2 * 2)
        out = F.relu(self.linlay0(out))
        out = F.relu(self.linlay1(out))
        out = self.linlay2(out)
        
        return F.softmax(out, dim=1)'''
class cnn(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.mxpul = torch.nn.MaxPool2d((3, 3), stride = 3)

        self.convlay0 = torch.nn.Conv2d(3, 32, 3, stride = 1)
        self.convlay1 = torch.nn.Conv2d(32, 32, 3, stride = 1)
        self.convlay2 = torch.nn.Conv2d(32, 64, 3, stride = 1)

        self.linlay0 = torch.nn.Linear(64 * 8 * 8, 1000)
        self.linlay1 = torch.nn.Linear(1000, 100)
        self.linlay2 = torch.nn.Linear(100, 10)
    
    def forward(self, x):

        out = F.relu(self.convlay0(x))
        out = F.relu(self.convlay1(out))
        out = self.mxpul(F.relu(self.convlay2(out)))
        out = out.view(-1, 64 * 8 * 8)
        out = F.relu(self.linlay0(out))
        out = F.relu(self.linlay1(out))
        out = self.linlay2(out)
        
        return F.softmax(out, dim=1)

In [6]:
net = cnn()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
net.to(device)

cnn(
  (mxpul): MaxPool2d(kernel_size=(3, 3), stride=3, padding=0, dilation=1, ceil_mode=False)
  (convlay0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (convlay1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (convlay2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (linlay0): Linear(in_features=4096, out_features=1000, bias=True)
  (linlay1): Linear(in_features=1000, out_features=100, bias=True)
  (linlay2): Linear(in_features=100, out_features=10, bias=True)
)

In [9]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum = 0.9)

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)
print(len(trainldr.dataset))

50000


In [11]:
epochs = 20

for epoch in range(epochs):
    start_time = time.time()
    mean_loss = 0
    for data, labels in trainldr:
        data, labels = Variable(data).to(device), Variable(labels).to(device)
        
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, labels)
        mean_loss += loss.data
        loss.backward()
        optimizer.step()
    print("number epoch: {} \t epoch execution time: {} \t  mean_loss: {}".format(
        epoch + 1, time.time() - start_time, mean_loss / len(trainldr.dataset)))

number epoch: 1 	 epoch execution time: 4.275771617889404 	  mean_loss: 0.023024555295705795
number epoch: 2 	 epoch execution time: 4.203607559204102 	  mean_loss: 0.023015407845377922
number epoch: 3 	 epoch execution time: 4.12800407409668 	  mean_loss: 0.022471321746706963
number epoch: 4 	 epoch execution time: 4.519702434539795 	  mean_loss: 0.021697569638490677
number epoch: 5 	 epoch execution time: 4.802273511886597 	  mean_loss: 0.021450934931635857
number epoch: 6 	 epoch execution time: 5.7396485805511475 	  mean_loss: 0.021229254081845284
number epoch: 7 	 epoch execution time: 5.972577333450317 	  mean_loss: 0.02093009650707245
number epoch: 8 	 epoch execution time: 5.799333095550537 	  mean_loss: 0.020772883668541908
number epoch: 9 	 epoch execution time: 6.098953723907471 	  mean_loss: 0.020583802834153175
number epoch: 10 	 epoch execution time: 6.0931079387664795 	  mean_loss: 0.020372958853840828
number epoch: 11 	 epoch execution time: 5.940214395523071 	  mean_lo

In [12]:
test_loss = 0
correct = 0
for data, labels in testldr:
    data, labels = Variable(data).to(device), Variable(labels).to(device)
    net_out = net(data)
    test_loss += criterion(net_out, labels).data
    for ind, t in enumerate(net_out):
        mxind1, mxind2 = 0, 1
        for ind2, x in enumerate(t):
            if t[ind2] > t[mxind1]:
                mxind2 = mxind1
                mxind1 = ind2
            elif ind2 != mxind1 and t[ind2] > t[mxind2]:
                mxind2 = ind2
        if labels[ind] == mxind1 or labels[ind] == mxind2:
            correct += 1
   

test_loss /= len(testldr.dataset)
print('mean_loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
       test_loss, correct, len(testldr.dataset),
       100. * correct / len(testldr.dataset)))

mean_loss: 0.0192, Accuracy: 7279/10000 (73%)
